# Examples of using velociraptor data

In [ ]:
#load matplotlib as inline so figures display within the notebook
%matplotlib inline
from matplotlib.pylab import *
rcParams['figure.figsize'] = (10,6)
rcParams['font.size'] = 18

In [ ]:
#load other useful packages
import numpy as np
import astropy as ap
import pynbody as pyn
#for general python stuff
import sys,os,string,time,re,struct
import math,operator
#for useful scipy stuff
from scipy.stats.mstats import mquantiles
from scipy.misc import comb
import scipy.interpolate as scipyinterp
import scipy.integrate as scipyint
import scipy.optimize as scipyopt
import scipy.special as scipysp
import itertools
#for useful mathematical tools
from sklearn.neighbors import NearestNeighbors
import scipy.spatial as spatial
#for multiprocessing
import multiprocessing as mp
#for stats
import emcee
#for plotting distributions
import corner

#to load specific functions defined in another python file
sys.path.append('./dir/to/VELOCIraptor-stf/stf/tools/')
import velociraptor_python_tools as vpt

## Loading Halos

In [ ]:
#define formats
ASCIIFORMAT=0
HDFFORMAT=2

#base filename 
inputfname="/outdir/snapshot_200.VELOCIraptor"

In [ ]:
#now we try loading halo property data
halopropdata,numhalos=ReadPropertyFile(inputfname,HDFFORMAT)

In [ ]:
#then load the particles associated with halos
haloparticledata=ReadParticleDataFile(inputfname,HDFFORMAT)

In [ ]:
#list the keys available
halopropdata.keys()
haloparticledata.keys()

## Plotting halo mass function

In [ ]:
#Set bin edges (log M)
NBINS=50
xlim=[5,15]
xedges=np.arange(xlim[0],xlim[1]+(xlim[1]-xlim[0])/float(NBINS),(xlim[1]-xlim[0])/float(NBINS))

#desired mass field
massfield=['Mass_200crit','Mass_tot', 'Mass_200mean']
massbindata=dict()
#make histograms while converting units to Msun and kpc
for key in massfield:
    massbindata[key],blah=np.histogram(np.log10(halopropdata["Mass_200crit"]),xedges)
    massbindata[key]=massbindata[key]+np.log10(halopropdata['Mass_to_Msun'])
    massbindata[key]=np.float64(massbindata[key])/np.power(halopropdata['BoxSize']/halopropdata['hval'],3.0)
#store mass value at which a halo is composed of 100 particles
vlinedata=halopropdata["Mass_tot"][np.where(halopropdata["npart"]==100)][0]

In [ ]:
colors=['red','blue','green']
vlabels=[r'$M_{200\rho_c}$',r'$M_{\textsc{tot}}$',r'$M_{200\rho_m}$']
for i in range(len(massfield)):
    plot(xbins,massbindata[massfield[i]],color=colors[i],label=vlabels[i])
ylabel(r'$dn/d\log M$ (kpc$^{-3}$)')
xlabel(r'$M$ [M$_\odot$]')
legend()